### Downloads

In [ ]:
!pip install chess python-lichess
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [825 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadc

### Imports and Setup

In [ ]:
import lichess.api
from lichess.format import SINGLE_PGN
from lichess.format import PGN, PYCHESS
from selenium import webdriver  # webdriver
from getpass import getpass     # getpass is used to receive password
from selenium.webdriver.chrome.options import Options # Chrome Options
from selenium.webdriver.common.by import By  # detect the page elements
from selenium.webdriver.support.wait import WebDriverWait# Driver delay
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time # Delay in code
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import json
# mounting google drive
from google.colab import drive
import os
import subprocess
drive.mount('/content/gdrive')
# importing other packages
import os
import datetime
import warnings
import shutil
from tqdm import tqdm
import time
import pandas as pd
warnings.filterwarnings("ignore")

Mounted at /content/gdrive


In [ ]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [ ]:
# File Directory Setup
cur_dir = os.getcwd()
# runs_dir = '/content/gdrive/MyDrive/Runs/'
run_help_dir = '/content/gdrive/MyDrive/runs_helper/'
# get run info
# player csv files
beginner_df = pd.read_csv('/content/gdrive/MyDrive/Batch Runs/unique_player_classical_beginner.csv')
inter_df = pd.read_csv('/content/gdrive/MyDrive/Batch Runs/unique_player_classical_intermediate.csv')
expert_df = pd.read_csv('/content/gdrive/MyDrive/Batch Runs/unique_player_classical_expert.csv')

In [ ]:
inter_df

,Players
0,Ardianch
1,Mohdjell79
2,PBdiamond
3,radhoose
4,Thinker100
...,...
2309,ReadySalted8625
2310,Stepa63
2311,VinodMittal
2312,Opmi


In [ ]:
#@title Login
gmail_username = 'b8739983@gmail.com' # @param {type:"string"}
gmail_password = 'hs0iaF3cON' # @param {type:"string"}
login_username = 'b8739983' # @param {type:"string"}
login_password = 'hs0iaF3cON' # @param {type:"string"}

In [ ]:
#@title Login
login_link = 'https://lichess.org/auth/magic-link/login/Yjg3Mzk5ODN8MTcxNDUzMjkwMjY1MnwxZjFjY2FlMmI2MWY3Zg==' # @param {type:"string"}

In [ ]:
login_flag = False # checks if your logged in or not
driver = driversetup()
# go to login link
driver.get(login_link)
# get html info of page to check if login successful
page_info = driver.page_source
if "Too many requests" in page_info or "404" in page_info:
  driver.get('https://lichess.org/auth/magic-link')
  driver.find_element("id", "form3-email").send_keys(gmail_username)
  driver.find_element(By.XPATH,"/html/body/div/main/form/div[3]/button").click()
  print("""
  Login was Unsuccessful:
  1. Check gmail for most recent email from lichess
  2. Update login_link variable with the magic_link in the email (if your not receiving a login link wait a couple seconds and rerun cell or try manually getting the link would need to fill out captcha)
  3. Run this cell again
  """)
else:
  print("Login Successful")
  login_flag = True

Login Successful


In [ ]:
folder = '/content/gdrive/MyDrive/Batch Runs/'
info_folders =  [os.path.join(folder,i) for i in os.listdir(folder) if 'info' in i]
begin_infos = [os.path.join(info_folders[0],i) for i in os.listdir(info_folders[0])]
inter_infos = [os.path.join(info_folders[1],i) for i in os.listdir(info_folders[1])]
expert_infos = [os.path.join(info_folders[2],i) for i in os.listdir(info_folders[2])]

### Beginner DF

In [ ]:
total_files = 0
for file in begin_infos:
  print(file)
  with open(file, 'r') as file_obj:
    contents = file_obj.readlines()
  print(len(contents))
  total_files += len(contents)

print(total_files)
print(beginner_df.shape[0])

# missing for 2000 - 2500, 1500 - 2000

/content/gdrive/MyDrive/Batch Runs/begin_info/info_500.txt
499
/content/gdrive/MyDrive/Batch Runs/begin_info/info_500_1000.txt
499
/content/gdrive/MyDrive/Batch Runs/begin_info/info_1000_1500.txt
499
/content/gdrive/MyDrive/Batch Runs/begin_info/info_1500_2000.txt
499
/content/gdrive/MyDrive/Batch Runs/begin_info/info_2000_2500.txt
464
2460
2465


In [ ]:
2465-2380+384

469

In [ ]:
clock_init_time = ["3 minutes", "10 minutes", "30 minutes"]

In [ ]:
max_games = 100

def iso_game_ids(search_page, total):
  ids = []
  ids_stored = 0
  flag = False
  num = search_page.count('game-row paginated')
  iso_page = search_page[search_page.find("game-row paginated"):search_page.rfind("game-row paginated")]
  if len(iso_page) == 0:
    iso_page = search_page[search_page.find("game-row paginated"):]
  for i in range(num):
    cur_game = iso_page[iso_page.find("game-row paginated"):iso_page.rfind("game-row paginated")]
    if len(cur_game) == 0:
      cur_game = search_page[search_page.find("game-row paginated"):]
    game_id = cur_game[cur_game.find("href")+7:cur_game.find("href")+15]
    iso_page = iso_page[iso_page.find("</article>")+10:]
    ids_stored+=1
    ids.append(game_id)
  return ids

def perform_search(player_name, init_time):
  driver.get(f'https://lichess.org/@/{player_name}/search')
  if 'This account is closed' in driver.page_source:
    return ''
  # init time - 3, 10, 30
  driver.find_element("id", "form3-clock_initMin").send_keys(init_time)
  driver.find_element("id", "form3-clock_initMax").send_keys(init_time)
  # inc time + 0
  driver.find_element("id", "form3-clock_incMin").send_keys("0 seconds")
  # input clock increment to
  driver.find_element("id", "form3-clock_incMax").send_keys("0 seconds")
  driver.find_element(By.XPATH, "/html/body/div[1]/main/div/form/table[2]/tbody/tr[12]/td/button").click()
  games_page = driver.page_source
  return games_page

# returns list of game ids depending on search, could be 0 if nothing
def get_search_list(player_name, duration):
  zero_flag = True
  games_page = perform_search(player_name, duration)
  num_games = 0
  # check search 10 times with delay of 3 seconds
  for i in range(15):
    games_page = perform_search(player_name, duration)
    if games_page.count('game-row paginated') == 0:
      time.sleep(0.1)
    else:
      zero_flag = False
      break
  if zero_flag:
    return []
  if games_page.count('game-row paginated') >= 100: # more games on page than max games
    game_ids = iso_game_ids(games_page, max_games)

  else: # less games on page than max games
    current_page = 0
    game_ids = []
    more_games = True if 'class="pager"' in games_page else False
    while 1:
      games_page = driver.page_source
      num_games += games_page.count('game-row paginated')
      more_games = True if 'class="pager"' in games_page else False
      game_ids = [i for i in game_ids if len(i) > 0]
      if len(game_ids) >= max_games:
        break
      if more_games:
        cur_page_ids = iso_game_ids(games_page, games_page.count('game-row paginated'))
        game_ids += cur_page_ids
        games_page = games_page[games_page.find('<div class="pager">'):]
        next_link = games_page[games_page.find('href')+6:games_page.find('</div>')-10]
        driver.get(next_link) # go to next page
        current_page+=1
      else:
        cur_page_ids = iso_game_ids(games_page, games_page.count('game-row paginated'))
        game_ids += cur_page_ids
        break

  game_ids = [i for i in game_ids if len(i) > 0]
  game_ids = game_ids[:max_games]
  return game_ids

In [ ]:
folders = ['begin_500','begin_500_1000','begin_1000_1500','begin_1500_2000','begin_2000_2500']
folders = [os.path.join('/content/gdrive/MyDrive/Batch Runs/',folder) for folder in folders]
file_names = ['info_500.txt','info_500_1000.txt','info_1000_1500.txt','info_1500_2000.txt','info_2000_2500.txt']
file_dirs = [os.path.join('/content/gdrive/MyDrive/Batch Runs/begin_info',file) for file in file_names]

In [ ]:
# missing for 2000 - 2500, 1500 - 2000 only 2

idx = -2
info_dir = file_dirs[idx]

with open(info_dir,'r') as file_obj:
  info_contents = file_obj.readlines()
total_scraped = len(info_contents)
if total_scraped < 499:
  last_user = info_contents[-1].split(" ")[0]
  # remove the last line
  info_contents = info_contents[:-1]
  with open(info_dir, 'w') as file_obj:
    for line in info_contents:
        file_obj.write(str(line))
# # - remove all pgn files from the folder
  folder = folders[idx]
  run_folder_path = folder
  folder_name_contents = folder.split("_")
  from_idx = int(folder_name_contents[-2]) if len(folder_name_contents) > 2 else 0
  to_idx = int(file_names[idx].split("_")[-1][:-4])
  # print(from_idx+total_scraped,to_idx, last_user)
  for add_idx in tqdm(range(from_idx+total_scraped, to_idx)):
    player_name = beginner_df.iloc[add_idx]['Players']
    print(player_name)
    meta_info_dir = info_dir
    ids_3 = get_search_list(player_name, clock_init_time[0])
    ids_10 = get_search_list(player_name, clock_init_time[1])
    if len(ids_3) == 0 and len(ids_10) == 0:
      with open(meta_info_dir, 'a') as outfile: outfile.write(f'{player_name} 0 0 0 \n')
      continue
    ids_30 = get_search_list(player_name, clock_init_time[2])
    # print("30 Minutes", len(ids_30), ids_30,"\n")
    if len(ids_3) >= 0:
      for game in ids_3:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_3.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    if len(ids_10) >= 0:
      for game in ids_10:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_10.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    if len(ids_30) >= 0:
      for game in ids_30:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_30.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    with open(meta_info_dir, 'a') as outfile: outfile.write(f'{player_name} {len(ids_3)} {len(ids_10)} {len(ids_30)} \n')

  0%|          | 0/5 [00:00<?, ?it/s]

Arcanis3332


 20%|██        | 1/5 [03:36<14:26, 216.52s/it]

Canban


 40%|████      | 2/5 [04:23<05:51, 117.05s/it]

...

 60%|██████    | 3/5 [08:13<05:36, 168.44s/it]

BlackHasAdvantage


 80%|████████  | 4/5 [16:18<04:53, 293.36s/it]

patnuri1


100%|██████████| 5/5 [25:14<00:00, 302.84s/it]


In [ ]:
beginner_df.shape

,Players
0,BaiHunor
1,lugokevin1
2,Digzou
3,IconN26
4,BillLuz
...,...
2460,Luvmagic
2461,kings00
2462,Biniam2
2463,harharmahadev2


In [ ]:
# missing for 2000 - 2500, 1500 - 2000 only 2

idx = -1
info_dir = file_dirs[idx]

with open(info_dir,'r') as file_obj:
  info_contents = file_obj.readlines()
total_scraped = len(info_contents)
if total_scraped < 469:
  last_user = info_contents[-1].split(" ")[0]
  # remove the last line
  info_contents = info_contents[:-1]
  with open(info_dir, 'w') as file_obj:
    for line in info_contents:
        file_obj.write(str(line))
# # - remove all pgn files from the folder
  folder = folders[idx]
  run_folder_path = folder
  folder_name_contents = folder.split("_")
  from_idx = int(folder_name_contents[-2]) if len(folder_name_contents) > 2 else 0
  to_idx = int(file_names[idx].split("_")[-1][:-4])
  # print(from_idx+total_scraped,to_idx, last_user)
  for add_idx in tqdm(range(from_idx+total_scraped, to_idx)):
    player_name = beginner_df.iloc[add_idx]['Players']
    print(player_name)
    meta_info_dir = info_dir
    ids_3 = get_search_list(player_name, clock_init_time[0])
    ids_10 = get_search_list(player_name, clock_init_time[1])
    if len(ids_3) == 0 and len(ids_10) == 0:
      with open(meta_info_dir, 'a') as outfile: outfile.write(f'{player_name} 0 0 0 \n')
      continue
    ids_30 = get_search_list(player_name, clock_init_time[2])
    # print("30 Minutes", len(ids_30), ids_30,"\n")
    if len(ids_3) >= 0:
      for game in ids_3:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_3.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    if len(ids_10) >= 0:
      for game in ids_10:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_10.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    if len(ids_30) >= 0:
      for game in ids_30:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_30.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    with open(meta_info_dir, 'a') as outfile: outfile.write(f'{player_name} {len(ids_3)} {len(ids_10)} {len(ids_30)} \n')

  0%|          | 0/36 [00:00<?, ?it/s]

dsd07


  3%|▎         | 1/36 [03:08<1:50:02, 188.64s/it]


IndexError: single positional indexer is out-of-bounds

In [ ]:
# check the final folder from 2000-2465
idx = -1
folder = folders[-1]
info_dir = os.path.join(folder,"info.txt")
with open(info_dir,'r') as file_obj:
  info_contents = file_obj.readlines()
total_scraped = len(info_contents)
if total_scraped < 465:
  # remove the last one since session might have terminated, just redo the last one from there (naive approach, remove from list and write again to the file)
  last_user = info_contents[-1].split(" ")[0]
  info_contents = info_contents[:-1]
  with open(info_dir, 'w') as file_obj:
    for line in info_contents:
        file_obj.write(str(line))
  # - remove all pgn files from the folder
  folder_contents = [for file in os.listdir(folder) if last_user in file]
  for file in folder_contents:
    file_dir = os.path.join(folder, file)
    os.remove(file_dir)
  # from the current idx finish to 500
  from_idx = int(folder_names[idx].split("_")[-2])
  to_idx = int(folder_names[idx].split("_")[-1])
  for add_idx in tqdm(from_idx+total_scraped-2, to_idx):
    player_name = beginner_df.iloc[add_idx]['Players']
    print(player_name)
    meta_info_dir = info_dir
    ids_3 = get_search_list(player_name, clock_init_time[0])
    ids_10 = get_search_list(player_name, clock_init_time[1])
    if len(ids_3) == 0 and len(ids_10) == 0:
      with open(meta_info_dir, 'a') as outfile: outfile.write(f'{player_name} 0 0 0 \n')
      continue
    ids_30 = get_search_list(player_name, clock_init_time[2])
    # print("30 Minutes", len(ids_30), ids_30,"\n")
    if len(ids_3) >= 0:
      for game in ids_3:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_3.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    if len(ids_10) >= 0:
      for game in ids_10:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_10.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    if len(ids_30) >= 0:
      for game in ids_30:
        cur_pgn = lichess.api.game(game, format=PGN)
        pgn_store_dir = os.path.join(run_folder_path,f'{player_name}_{game}_30.pgn')
        with open(pgn_store_dir, 'w') as outfile: outfile.write(cur_pgn)
    with open(meta_info_dir, 'a') as outfile: outfile.write(f'{player_name} {len(ids_3)} {len(ids_10)} {len(ids_30)} \n')


### Run Parameters